### 0 shot QA on french traffic laws

In this project I'll try to use the Code de la route (french road code) data and the model CamembertForQa for the question answering.

1. I used a pinecone database to store our embeddings representing the content of the code de la route aswell as 'dangvantuan/sentence-camembert-base' sentence transformer to generate the embeddings.
2. We then embed our question using the same transformer and compare to the vectors in pinecone using dotproduct and we get the k-neighbor closest ones.
3. We hope this vectors represent related law articles to our question.
4. We use these vectors as context and feed it with the question to a CamembertForQa model.



In [ ]:
pip install sentence_transformers  pinecone-client

In [2]:
import json

#Response from legifrance api to get code de la route data
f = open('code_de_la_route_json.json', encoding="utf8")

data = json.load(f)

This is the most important step since we are trying 0shot QA. We need to find a way to split the articles such as:
 - The sentences are small enough way that the embedding encapsulate their meaning
 - Not too small so that the meaning is kept in each sentences

On my first try, i encoded the whole article into a single vector wich resulted in poor retrieval peformance.
I then tried splitting at '.'. But this has 2 problems:
 - There are alot of '.' that don't represent an end of sentence (ex: "Par dérogation aux dispositions de l'article L. 121-1")
 - Some idea in the article can only be understood when taking the whole context in account and not the sentence alone.
   
I came up with these potential solutions:

 - For the first problem, we can just ignore '.' that dont represent end of sentence using a regex
 - To keep the meaning of the article we could for example encode sentences 2 by 2 with overlaps, to try to keep the meaning in the embedding.
 - Another solution could be to keep our single sentence representing vectors, but use the whole article they are from, as context for our RAG

In [ ]:
import re

CLEANR = re.compile('<.*?>|\t|\n') 

def cleanhtml(raw_html):
    cleantext = re.sub(CLEANR, ' ', raw_html)
    return cleantext

def split_article(content):
    # Utilisation d'une expression régulière pour découper la chaîne
    # en gardant les points suivis par un espace puis un nombre ensemble
    result = re.split(r'\. (?=\D)', content)
    
    # Nettoyage des espaces en trop autour des éléments
    result = [phrase.strip() for phrase in result if phrase.strip()]
    
    # Affichage du tableau résultant
    return result


def process_row(row_data, chapitre, articleNum): 
    clean_content = cleanhtml(row_data['content'])
    #Split sentences in the article for better embedding on pinecone
    split_content = split_article(clean_content)
    for sentence in split_content:
        processed_data.append({'chapitreNom': chapitre, 'articleNum': articleNum, 'content': sentence})
                
processed_data = []
#Process data from code de la route
for partie in data['sections']:
    for livre in partie['sections']:
        for titre in livre['sections']:
            for chapitre in titre['sections']:
                if len(chapitre['articles']) == 0:
                    for section in chapitre['sections']:
                        for articleIn in section['articles']:
                            print(articleIn['num'])
                            process_row(articleIn, chapitre['title'], articleIn['num'])
                else: 
                    for article in chapitre['articles']:
                        process_row(article, chapitre['title'], article['num'])


In [114]:
import pandas as pd

df = pd.DataFrame(processed_data)
df

,chapitreNom,articleNum,content
0,Chapitre 1er : Responsabilité pénale.,L121-1,Le conducteur d'un véhicule est responsable pé...
1,Chapitre 1er : Responsabilité pénale.,L121-1,"Toutefois, lorsque le conducteur a agi en qual..."
2,Chapitre 1er : Responsabilité pénale.,L121-2,Par dérogation aux dispositions de l'article ...
3,Chapitre 1er : Responsabilité pénale.,L121-2,Dans le cas où le véhicule était loué à un tie...
4,Chapitre 1er : Responsabilité pénale.,L121-2,"Dans le cas où le véhicule a été cédé, cette r..."
...,...,...,...
4787,Chapitre 1er : Responsabilité pénale,A121-3,antai
4788,Chapitre 1er : Responsabilité pénale,A121-3,fr ” la copie du certificat d'immatriculation ...
4789,Chapitre 1er : Responsabilité pénale,A121-3,"IV.- Dans tous les cas, un accusé d'enregistre..."
4790,Chapitre 1er : Responsabilité pénale,A121-3,Ce document peut être téléchargé ou imprimé pa...


In [116]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

retriever = SentenceTransformer(
    "dangvantuan/sentence-camembert-base",
    device=device
)

In [76]:
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: CamembertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [117]:
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key="<<PINECONE_API_KEY>>")


In [118]:
index_name = "traffic-question-answering-2"

#Need to be the same dimension as the output of our sentence transformer
if index_name not in pc.list_indexes():
    pc.create_index(
        index_name,
        dimension=768,
        metric='dotproduct',
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ) 
    )
index = pc.Index(index_name)

In [119]:

from tqdm.auto import tqdm

# we will use batches of 1
batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(df))
    # extract batch
    batch = df.iloc[i:i_end]
    # generate embeddings for batch
    emb = retriever.encode(batch["content"].tolist()).tolist()
    # get metadata
    meta = batch.to_dict(orient="records")
    # create unique IDs
    ids = [f"{idx}" for idx in range(i, i_end)]
    # add all to upsert list
    to_upsert = list(zip(ids, emb, meta))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

100%|██████████| 75/75 [00:43<00:00,  1.71it/s]


{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 4736}},
 'total_vector_count': 4736}

In [120]:
from transformers import AutoTokenizer, CamembertForQuestionAnswering

# load bart tokenizer and model from huggingface
tokenizer = AutoTokenizer.from_pretrained('CATIE-AQ/QAmembert')
model = CamembertForQuestionAnswering.from_pretrained('CATIE-AQ/QAmembert')

In [121]:
def query_pinecone(query, top_k):
    # generate embeddings for the query
    xq = retriever.encode([query]).tolist()
    # search pinecone index for context passage with the answer
    xc = index.query(vector=xq, top_k=top_k, include_metadata=True)
    return xc

def format_query(query, context):
    # extract passage_text from Pinecone search result and add the tag
    context = [f" {m['metadata']['content']}" for m in context]
    # concatinate all context passages
    context = " ".join(context)
    # contcatinate the query and context passages

    return query, context

In [122]:
def format_query_get_article(query, context):
    article_nums = [f"{m['metadata']['articleNum']}" for m in context]
    article_keys = list(set(article_nums))
    articles = [ df.loc[df['articleNum'] == key]['content'].str.cat(sep=' ') for key in article_keys ]
    context = "".join(articles)
    return query, context

In [123]:
from transformers import pipeline

#Using pretrained Camembert model for QUestion answering
qa_engine = pipeline(
    "question-answering",
    model="CATIE-AQ/QAmembert",
    tokenizer="CATIE-AQ/QAmembert"
)

def generate_answer(query, context):
    result = qa_engine(
        context=context,
        question=query
    )
    return result

In [124]:
query = "Quel est l'âge minimum faire la conduite encadrée ?"
result = query_pinecone(query, top_k=3)

#Get related conext from pinecone database
query, context = format_query_get_article(query, result["matches"])

result

{'matches': [{'id': '4140',
              'metadata': {'articleNum': 'R414-4',
                           'chapitreNom': 'Chapitre IV : Croisement et '
                                          'dépassement',
                           'content': '- Avant de dépasser, tout conducteur '
                                      "doit s'assurer qu'il peut le faire sans "
                                      'danger'},
              'score': 5.13207722,
              'values': []},
             {'id': '1065',
              'metadata': {'articleNum': 'R211-1',
                           'chapitreNom': 'Chapitre Ier : Formation à la '
                                          'conduite et à la sécurité routière',
                           'content': 'La date limite de validité est inscrite '
                                      'sur le titre de conduite'},
              'score': 5.02681398,
              'values': []},
             {'id': '82',
              'metadata': {'articleNum': 'L211-

In [125]:
#1st question
print(generate_answer(query, context)['answer'])

#2nd question
query = "Depuis combien de temps doit-on être titulaire pour être accompagnateur de la conduite supervisée ?"
result = query_pinecone(query, top_k=3)
query, context = format_query_get_article(query, result["matches"])
print(generate_answer(query, context)['answer'])

seize ans
cinq ans


Using the articles from the 3 vectors closest to the question in the database.
We can observe 2 limitations

1. Our sentence transformer creating our embedding in pinecone fails to return context properly when articles are mentionned in each others.
   This is likely due to the way the code relations between article are abstractly described. Wich is probably something our sentence transformer is not well tuned for
2. The CATIE-AQ/QAmembert model manages to retrieve the informations in the context but the output is quite small and doesn't allow for a detailed response

In the example above we ask 2 questions

1. What is the minimum age to do accompanied driving
2. How much year of driving experience is needed supervise accompanied driving 

The model answer correctly for these 2 questions


In [126]:
df.loc[df['articleNum'] == 'R211-5']['content'].str.cat(sep=' ') 

"I.-L'apprentissage dit anticipé de la conduite est un apprentissage particulier dispensé en vue de l'obtention de la catégorie B du permis de conduire Cet apprentissage ne peut être effectué après annulation ou invalidation du permis de conduire II.-Il comprend deux périodes :   1° Une période de formation initiale dans un établissement ou une association agréés au titre de l'article  L. 213-1  ou  L. 213-7 Cette formation initiale est validée si l'élève conducteur a réussi l'épreuve théorique générale de l'examen du permis de conduire ou détient une catégorie du permis de conduire obtenue depuis cinq ans au plus, et s'il réussit l'évaluation réalisée par l'enseignant de la conduite à la fin de cette période ;   2° Une période d'apprentissage en conduite accompagnée sous la surveillance constante et directe d'un accompagnateur titulaire depuis au moins cinq ans sans interruption du permis de conduire de la catégorie B Cette période commence par un rendez-vous pédagogique préalable ent